In [4]:
import pyodbc

import numpy as np
import pandas as pd

from sqlalchemy import create_engine

from FlaskApp.transform import *

In [5]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [50]:
df_applicant = pd.DataFrame(engine.execute(
    f"""
    SELECT Applicant.ApplicantID, Applicant.Dob, Applicant.ExpectedSalary
    FROM Applicant
    """
)).set_index(['ApplicantID'])

df_applicant_education = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantEducation.ApplicantID, ApplicantEducation.DateStart, ApplicantEducation.DateEnd, EducationLevel.EducationLevelName, Major.MajorName
    FROM ((ApplicantEducation
    RIGHT JOIN EducationLevel ON ApplicantEducation.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN Major ON ApplicantEducation.MajorID = Major.MajorID)
    """
))

df_applicant_experience = pd.DataFrame(engine.execute(
    f"""
    SELECT ApplicantExperience.ApplicantID, ApplicantExperience.DateFrom, ApplicantExperience.DateTo, ApplicantExperience.Industry, ApplicantExperience.Position, ApplicantExperience.Salary
    FROM (ApplicantExperience
    LEFT JOIN Pipeline ON ApplicantExperience.ApplicantID = Pipeline.ApplicantID)
    """
))

df_job = pd.DataFrame(engine.execute(
    """
    SELECT Job.JobID, Job.UsiaMax, Job.SalaryMin, Job.SalaryMax, City.Name AS CityName, Province.Name AS ProvinceName, EducationLevel.EducationLevelName, Major.MajorName, Job.DriverLicenseType, Job.UsingGlasses, Job.Gender, Job.MaritalStatus, Job.JobTitle, FunctionPosition.FunctionPositionName, Job.Description, Job.Requirement
    FROM (((((Job
    RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
    RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
    RIGHT JOIN City ON Job.CityID = City.CityID)
    RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
    RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
    WHERE JobStatus='Publish'
    """
))

In [78]:
df_applicant.fillna('', inplace=True)

df_applicant['Age'] = pd.to_datetime(
    df_applicant.Dob.map(pick_date).apply(lambda x: filter_date(x, 1958, 2006))
).map(get_age)

df_applicant.drop(columns=['Dob'], inplace=True)

df_applicant.Age = df_applicant.Age.fillna(0).astype(int)

In [82]:
df_applicant_education = df_applicant_education.fillna('')
df_applicant_education.DateStart = pd.to_datetime(
    df_applicant_education.DateStart.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_education.DateEnd = pd.to_datetime(
    df_applicant_education.DateEnd.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_education = df_applicant_education[~(df_applicant_education.DateStart.isna()) & ~(df_applicant_education.DateEnd.isna())]
df_applicant_education = df_applicant_education.sort_values('DateStart').groupby(['ApplicantID']).agg('last')

df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)

In [85]:
df_applicant_education

,EducationLevelName,MajorName
ApplicantID,,
1,SMA,SMA SEGALA JURUSAN
3,S1,SEMUA JURUSAN
4,S1,TEKNIK INFORMATIKA
8,S1,SEMUA JURUSAN
13,D4,PSIKOLOGI
...,...,...
44923,S1,TEKNIK SIPIL
44925,S1,KOMUNIKASI
44928,SMA,SMA SEGALA JURUSAN


In [69]:
df_applicant_experience = df_applicant_experience.drop_duplicates()
df_applicant_experience = df_applicant_experience[df_applicant_experience.Salary != 0]
df_applicant_experience.Salary = df_applicant_experience.Salary.apply(lambda x: x*1_000_000 if x <= 20 else x)
df_applicant_experience.Salary = df_applicant_experience.Salary.apply(lambda x: x*1_00_000 if 20 < x < 100 else x)
df_applicant_experience.Salary = df_applicant_experience.Salary.apply(lambda x: x*1_000 if 1000 <= x < 10_000 else x)
df_applicant_experience = df_applicant_experience[df_applicant_experience.Salary > 100_000]

In [87]:
df_applicant_experience.fillna('', inplace=True)

In [89]:
df_applicant_experience.DateFrom = pd.to_datetime(
    df_applicant_experience.DateFrom.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_experience.DateTo = pd.to_datetime(
    df_applicant_experience.DateTo.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
)

df_applicant_experience = df_applicant_experience[~(df_applicant_experience.DateFrom.isna()) & ~(df_applicant_experience.DateTo.isna())]

In [97]:
df_merged = pd.merge(df_applicant_experience, df_applicant, on=['ApplicantID'])
df_merged = pd.merge(df_merged, df_applicant_education, on=['ApplicantID'])

In [102]:
df_merged.set_index(['ApplicantID'], inplace=True)

In [104]:
df_merged.to_csv('data/salary_dataset.csv')